# BUILD A UNIVERSE OF HIGH-VOLATILITY STOCKS

<img src="https://github.com/Portman-Dixon/alpha_combination/blob/main/images/logo.png" alt="Logo" width="200" height="200">

# Summary

1. Goal
2. Import libraries
3. Extract the ticker name of each symbol from our universe of s&p500 stock's csv files
4. Import ohlcv data of each ticker and put them into a single dataframe
5. Remove the tickers that were not traded during all the lenght of the dataset
6. Store in a new list, only the tickers that were traded during all the lenght of the dataset
7. Separate the dataset into a train set and a test set
8. Compute volatility and autocorrelation of stocks on in sample data
9. Extract the stocks with positive autocorrelation and the stocks with negative autocorrelation
10. Extract the 30 stocks with the highest volatility
11. Download ohlcv data for these stocks
12. Export data to csv format

# 1. Goal

- Extract a universe of 30 stocks with positive auto-correlation and high volatility (for testing a momentum strategy)
- Extract a universe of 30 stocks with negative auto-correlation and high volatility (for testing a mean reversion strategy)

# 2. Import libraries

In [1]:
# Library for data handling
import pandas as pd
import numpy as np

# Libraries for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Library for fetching datas
import yfinance as yf

# Divers
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import pathlib
import statsmodels.api as sm

# 3. Extract the ticker name of each symbol from our universe of s&p500 stock's csv files

In [2]:
# Define a path toward the directory containing the csv files of each ticker
path = '/Path/toward/the/directory/where/your/.csv/files/are/stored/'

# Extract the tickers names and store them into a variable
selected_tickers = [filename.stem for filename in pathlib.Path(path).glob('*.csv')]
print(len(selected_tickers))
print(selected_tickers)

503
['CSCO', 'UAL', 'TROW', 'ISRG', 'NVR', 'TPR', 'DVN', 'CE', 'MRO', 'BA', 'VRTX', 'BRK.B', 'GILD', 'EQIX', 'TER', 'PODD', 'MDT', 'V', 'QRVO', 'A', 'FOX', 'FLT', 'MO', 'CTRA', 'SWKS', 'ENPH', 'MCHP', 'CDNS', 'MSCI', 'CHTR', 'EIX', 'KDP', 'BBY', 'GEN', 'WBA', 'LVS', 'HCA', 'AJG', 'DTE', 'C', 'T', 'CF', 'MGM', 'HUM', 'CBOE', 'CFG', 'APH', 'SYY', 'MSI', 'FCX', 'ADM', 'OGN', 'LH', 'LNT', 'BAC', 'LNC', 'PSX', 'GPN', 'PPG', 'TECH', 'IRM', 'IQV', 'ESS', 'WBD', 'HAL', 'STZ', 'DXC', 'BG', 'PARA', 'ADI', 'F', 'ADBE', 'STLD', 'CPRT', 'TDG', 'TFX', 'ULTA', 'ARE', 'SYK', 'CB', 'TSN', 'GNRC', 'PEP', 'PEG', 'NOW', 'LLY', 'COST', 'REG', 'NWS', 'LOW', 'MDLZ', 'BKNG', 'ZBRA', 'FMC', 'XEL', 'AIZ', 'MET', 'FTV', 'DLR', 'ACGL', 'XRAY', 'FAST', 'TJX', 'SNA', 'MPC', 'BR', 'D', 'MRK', 'STX', 'NOC', 'BXP', 'KHC', 'IPG', 'UNP', 'ALLE', 'ABBV', 'CDAY', 'ORCL', 'ECL', 'ETR', 'EBAY', 'SBUX', 'IR', 'AMT', 'INTU', 'DPZ', 'PAYC', 'CMA', 'PG', 'CAT', 'ODFL', 'MCD', 'MNST', 'AMZN', 'GEHC', 'INTC', 'PNR', 'GLW', 'BDX',

# 4. Import ohlcv data of each ticker and put them into a single dataframe

In [3]:
prices = []

# Iterate through the list of symbols to import their respective ohlc datas and to rename the columns of 
# their dataframes. Store the dataframes into the empty list 'prices'
for symbol in selected_tickers:
    
    df = pd.read_csv(f'{path}/{symbol}.csv')[['Date','Open','High','Low','Close']] # Import data into a dataframe
    df = df.set_index('Date') # Use the column Date as index
    df.columns = [f'{symbol}_o', f'{symbol}_h', f'{symbol}_l', f'{symbol}_c'] # Rename columns
    prices.append(df) # Store the dataframes into the list 'prices'
    
prices_df = pd.concat(prices, axis=1, sort=True) # Concatenate the dataframes to have a single dataframe

# 5. Remove the tickers that were not traded during all the lenght of the dataset

In [4]:
prices_df = prices_df.dropna(axis=1) # Remove columns containing NaN values
prices_df

,CSCO_o,CSCO_h,CSCO_l,CSCO_c,TROW_o,TROW_h,TROW_l,TROW_c,ISRG_o,ISRG_h,...,NWL_l,NWL_c,BLK_o,BLK_h,BLK_l,BLK_c,PPL_o,PPL_h,PPL_l,PPL_c
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,17.209999,17.490000,17.180000,17.450001,36.185001,36.584999,35.555000,36.494999,13.210000,13.210000,...,23.389999,23.850000,109.480003,109.529999,105.739998,109.000000,27.588068,28.016512,27.457672,27.941999
2006-01-04,17.480000,17.930000,17.459999,17.850000,36.494999,37.009998,36.494999,36.660000,12.914444,13.564444,...,23.700001,24.059999,109.250000,109.739998,108.660004,109.180000,28.128281,28.193478,27.429729,28.184164
2006-01-05,17.940001,18.480000,17.930000,18.350000,36.810001,36.910000,36.415001,36.630001,13.775556,13.777778,...,23.920000,24.150000,109.430000,109.430000,107.830002,108.000000,27.951315,28.212107,27.588068,27.662580
2006-01-06,18.510000,18.879999,18.469999,18.770000,36.750000,37.150002,36.639999,36.930000,13.081111,13.575556,...,23.900000,24.010000,107.790001,110.419998,107.790001,110.419998,27.941999,28.035139,27.774347,27.802290
2006-01-09,18.969999,19.110001,18.920000,19.059999,37.080002,37.459999,36.775002,37.430000,13.364444,13.888889,...,23.879999,24.280001,111.790001,112.489998,110.510002,111.699997,27.774347,27.979256,27.681208,27.979256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-20,51.939999,52.080002,51.470001,51.549999,112.940002,113.669998,111.860001,112.449997,328.000000,329.679993,...,8.200000,8.270000,695.690002,696.460022,687.500000,690.950012,26.910000,26.959999,26.639999,26.700001
2023-06-21,51.380001,51.400002,50.820000,50.849998,111.610001,112.150002,110.199997,110.239998,327.829987,327.829987,...,8.070000,8.100000,690.719971,696.150024,686.400024,689.039978,26.580000,26.950001,26.290001,26.889999
2023-06-22,50.650002,51.139999,50.430000,51.119999,109.820000,109.820000,106.480003,106.559998,323.690002,328.279999,...,7.830000,7.920000,684.409973,684.510010,675.369995,679.380005,26.959999,27.000000,26.540001,26.719999


# 6. Store in a new list, only the tickers that were traded during all the lenght of the dataset

In [5]:
cleaned_tickers_list = set()

def clean_tickers_name(column):
    
    ticker_name = column.split("_")[0] # Extract the ticker name before the underscore
    # Check if the column name is not 'doublon' and the cleaned name is not in the list
    if column != "doublon" and ticker_name not in cleaned_tickers_list:
        cleaned_tickers_list.add(ticker_name)
        return ticker_name
    return None

# Call the function for each column and filter out None values
selected_tickers2 = list(filter(None, map(clean_tickers_name, prices_df.columns)))
print(len(selected_tickers2))
print(selected_tickers2)

416
['CSCO', 'TROW', 'ISRG', 'NVR', 'TPR', 'DVN', 'CE', 'MRO', 'BA', 'VRTX', 'GILD', 'EQIX', 'TER', 'MDT', 'A', 'MO', 'CTRA', 'SWKS', 'MCHP', 'CDNS', 'EIX', 'BBY', 'GEN', 'WBA', 'LVS', 'AJG', 'DTE', 'C', 'T', 'CF', 'MGM', 'HUM', 'APH', 'SYY', 'MSI', 'FCX', 'ADM', 'LH', 'LNT', 'BAC', 'LNC', 'GPN', 'PPG', 'TECH', 'IRM', 'ESS', 'WBD', 'HAL', 'STZ', 'DXC', 'BG', 'PARA', 'ADI', 'F', 'ADBE', 'STLD', 'CPRT', 'TFX', 'ARE', 'SYK', 'CB', 'TSN', 'PEP', 'PEG', 'LLY', 'COST', 'REG', 'LOW', 'MDLZ', 'BKNG', 'ZBRA', 'FMC', 'XEL', 'AIZ', 'MET', 'DLR', 'ACGL', 'XRAY', 'FAST', 'TJX', 'SNA', 'D', 'MRK', 'STX', 'NOC', 'BXP', 'IPG', 'UNP', 'ORCL', 'ECL', 'ETR', 'EBAY', 'SBUX', 'AMT', 'INTU', 'DPZ', 'CMA', 'PG', 'CAT', 'ODFL', 'MCD', 'MNST', 'AMZN', 'INTC', 'PNR', 'GLW', 'BDX', 'CSGP', 'PWR', 'BBWI', 'DXCM', 'EXR', 'WELL', 'HOLX', 'EXPD', 'TXN', 'FI', 'SJM', 'TMO', 'OXY', 'RL', 'CCI', 'MMM', 'MOS', 'HSY', 'JNPR', 'DHI', 'ED', 'ES', 'ADSK', 'GL', 'IP', 'EXPE', 'KO', 'PCAR', 'RVTY', 'WDC', 'NEE', 'UPS', 'ELV',

# 7. Separate the dataset into a train set and a test set

In [11]:
split = int(len(prices_df)*0.5)
is_df = prices_df.iloc[:split]
oos_df = prices_df.iloc[split:]

print('length is_df:', len(is_df))
print('length oos_df:', len(oos_df))

length is_df: 2199
length oos_df: 2200


In [13]:
is_df

,CSCO_o,CSCO_h,CSCO_l,CSCO_c,TROW_o,TROW_h,TROW_l,TROW_c,ISRG_o,ISRG_h,...,NWL_l,NWL_c,BLK_o,BLK_h,BLK_l,BLK_c,PPL_o,PPL_h,PPL_l,PPL_c
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,17.209999,17.490000,17.180000,17.450001,36.185001,36.584999,35.555000,36.494999,13.210000,13.210000,...,23.389999,23.850000,109.480003,109.529999,105.739998,109.000000,27.588068,28.016512,27.457672,27.941999
2006-01-04,17.480000,17.930000,17.459999,17.850000,36.494999,37.009998,36.494999,36.660000,12.914444,13.564444,...,23.700001,24.059999,109.250000,109.739998,108.660004,109.180000,28.128281,28.193478,27.429729,28.184164
2006-01-05,17.940001,18.480000,17.930000,18.350000,36.810001,36.910000,36.415001,36.630001,13.775556,13.777778,...,23.920000,24.150000,109.430000,109.430000,107.830002,108.000000,27.951315,28.212107,27.588068,27.662580
2006-01-06,18.510000,18.879999,18.469999,18.770000,36.750000,37.150002,36.639999,36.930000,13.081111,13.575556,...,23.900000,24.010000,107.790001,110.419998,107.790001,110.419998,27.941999,28.035139,27.774347,27.802290
2006-01-09,18.969999,19.110001,18.920000,19.059999,37.080002,37.459999,36.775002,37.430000,13.364444,13.888889,...,23.879999,24.280001,111.790001,112.489998,110.510002,111.699997,27.774347,27.979256,27.681208,27.979256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-09-22,25.190001,25.190001,24.860001,24.969999,80.070000,80.080002,79.169998,79.190002,51.782223,51.867779,...,34.759998,34.910000,331.000000,331.000000,326.149994,327.609985,31.239157,31.248470,30.875910,31.034248
2014-09-23,24.870001,25.030001,24.670000,24.700001,79.089996,79.650002,78.940002,79.040001,50.391109,50.873333,...,34.700001,35.009998,326.250000,328.559998,324.290009,325.059998,31.015619,31.090132,30.838654,30.838654
2014-09-24,24.730000,25.040001,24.629999,24.980000,79.349998,79.739998,79.019997,79.690002,50.911110,51.924442,...,34.470001,34.669998,325.519989,329.420013,325.000000,329.000000,30.875910,30.903852,30.372953,30.531292


# 8. Compute volatility and autocorrelation of stocks on in sample data

In [15]:
def calculate_autocorrelation_volatility(df):
    # Calculate the daily returns for each stock using the 'close' columns
    df_close = df[[col for col in df.columns if col.endswith('_c')]]
    df_returns = df_close.pct_change().dropna()

    # Create a new DataFrame to store the autocorrelation coefficients and volatility for each stock
    autocorrelation_df = pd.DataFrame()

    # Loop through each stock and calculate the autocorrelation coefficient and volatility
    for ticker in set(col.split('_')[0] for col in df_returns.columns):
        returns_col = f"{ticker}_c"
        prev_returns_col = f"{ticker}_c" + '_lagged'  # Add '_lagged' to create the lagged returns column

        # Create the lagged returns column (shifted by one day)
        df_returns[prev_returns_col] = df_returns[returns_col].shift(1)

        # Remove NaN rows introduced by the shift
        df_returns.dropna(inplace=True)

        # Perform linear regression of current day's returns against previous day's returns
        model = sm.OLS(df_returns[returns_col], sm.add_constant(df_returns[prev_returns_col]))
        results = model.fit()

        # Get the autocorrelation coefficient (regression coefficient)
        autocorrelation_coefficient = results.params[prev_returns_col]

        # Calculate the volatility (standard deviation of daily returns)
        volatility = df_returns[returns_col].std()

        # Store the autocorrelation coefficient and volatility in the new DataFrame
        autocorrelation_df.loc[ticker, 'Autocorrelation'] = autocorrelation_coefficient
        autocorrelation_df.loc[ticker, 'Volatility'] = volatility

    return autocorrelation_df

autocorrelation_volatility_df = calculate_autocorrelation_volatility(is_df)
print(autocorrelation_volatility_df)

      Autocorrelation  Volatility
WELL        -0.206429    0.020403
GL          -0.134881    0.022545
DD          -0.049304    0.024412
ALL         -0.109752    0.023282
RSG         -0.030115    0.016791
...               ...         ...
MAA         -0.198579    0.027121
BKNG        -0.093792    0.028379
ESS         -0.191846    0.024764
HSY         -0.104227    0.014292
TDY         -0.106052    0.022065

[416 rows x 2 columns]


# 9. Extract the stocks with positive autocorrelation and the stocks with negative autocorrelation

In [16]:
positive_autocorrelation_df = autocorrelation_volatility_df[autocorrelation_volatility_df['Autocorrelation'] > 0]
negative_autocorrelation_df = autocorrelation_volatility_df[autocorrelation_volatility_df['Autocorrelation'] < 0]

print(len(positive_autocorrelation_df))
print(len(negative_autocorrelation_df))

60
356


# 10. Extract the 30 stocks with the highest volatility

In [17]:
top_30_posit_autocor = positive_autocorrelation_df.nlargest(30, 'Volatility')
top_30_negat_autocor = negative_autocorrelation_df.nlargest(30, 'Volatility')

print(top_30_posit_autocor)
print('')
print(top_30_negat_autocor)

      Autocorrelation  Volatility
AIG          0.145302    0.054396
AAL          0.030687    0.050370
LVS          0.068147    0.048678
FITB         0.003055    0.046090
MGM          0.119993    0.045871
RF           0.000799    0.042801
MS           0.005726    0.042535
C            0.053126    0.041812
DXCM         0.021108    0.040353
LEN          0.047150    0.039166
NFLX         0.049814    0.036632
PHM          0.017601    0.036631
LYV          0.030100    0.036546
AMD          0.007983    0.036487
MU           0.035840    0.036469
DHI          0.027930    0.036249
URI          0.088116    0.035828
RCL          0.030380    0.035698
WYNN         0.037843    0.035091
MOS          0.003260    0.033732
F            0.079799    0.031919
TTWO         0.007510    0.031908
CE           0.034037    0.030779
FFIV         0.000275    0.030173
ILMN         0.039737    0.029531
ISRG         0.016677    0.028851
VLO          0.002650    0.028792
KMX          0.012877    0.028163
BBY          0

In [18]:
# Store the tickers into a list
stocks1 = top_30_posit_autocor.index
stocks1 = stocks1.to_list()

stocks2 = top_30_negat_autocor.index
stocks2 = stocks2.to_list()

print('Stocks with positive autocorrelation and high volatility')
print(stocks1)
print('')
print('Stocks with negative autocorrelation and high volatility')
print(stocks2)

Stocks with positive autocorrelation and high volatility
['AIG', 'AAL', 'LVS', 'FITB', 'MGM', 'RF', 'MS', 'C', 'DXCM', 'LEN', 'NFLX', 'PHM', 'LYV', 'AMD', 'MU', 'DHI', 'URI', 'RCL', 'WYNN', 'MOS', 'F', 'TTWO', 'CE', 'FFIV', 'ILMN', 'ISRG', 'VLO', 'KMX', 'BBY', 'WY']

Stocks with negative autocorrelation and high volatility
['HIG', 'HBAN', 'CBRE', 'LNC', 'BAC', 'KEY', 'PFG', 'ZION', 'INCY', 'PRU', 'PLD', 'HST', 'AXON', 'VRTX', 'STLD', 'MKTX', 'TXT', 'COF', 'ALGN', 'MET', 'REGN', 'KIM', 'STT', 'WFC', 'FCX', 'SWKS', 'ICE', 'IVZ', 'RJF', 'STX']


# 11. Download ohlcv data for these stocks

In [19]:
data1 = {}

for ticker in stocks1:
    
    d1 = yf.download(ticker, start='2006-01-01', end='2023-07-05')
    data1[ticker] = d1
    
print('')
    
data2 = {}

for ticker in stocks2:
    
    d2 = yf.download(ticker, start='2006-01-01', end='2023-07-05')
    data2[ticker] = d2

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [23]:
print(data1)
print('')
print(data2)

{'AIG':                    Open         High          Low        Close   Adj Close  \
Date                                                                         
2006-01-03  1378.800049  1393.199951  1336.800049  1392.400024  886.258362   
2006-01-04  1395.800049  1407.000000  1385.199951  1394.400024  887.531128   
2006-01-05  1398.000000  1401.000000  1388.599976  1396.199951  888.676514   
2006-01-06  1400.000000  1405.400024  1388.000000  1402.199951  892.495789   
2006-01-09  1397.000000  1399.800049  1391.400024  1395.599976  888.294800   
...                 ...          ...          ...          ...         ...   
2023-06-27    55.540001    56.779999    55.400002    56.450001   56.450001   
2023-06-28    56.240002    56.520000    55.680000    56.459999   56.459999   
2023-06-29    56.590000    57.400002    56.549999    57.200001   57.200001   
2023-06-30    57.910000    57.950001    57.349998    57.540001   57.540001   
2023-07-03    57.549999    58.439999    57.549999    57.

# 12. Export data to csv format

In [13]:
# Export ohlcv data of each ticker in csv format
for ticker, df in data1.items():
    df.to_csv(f'{ticker}.csv')
    
for ticker, df in data2.items():
    df.to_csv(f'{ticker}.csv')